In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import multiprocessing as mul
from scipy import stats
import pickle
from scipy.integrate import quad
from corner import corner
import pandas as pd
from scipy.stats import gaussian_kde
import dynesty as dyn

In [2]:
all_BE = {}
for file in os.listdir('./outputs/BF/'):
    be = []
    with open('./outputs/BF/'+file, 'rb') as f:
        data = np.loadtxt(f, delimiter=',')
        # be = np.exp(data[:,0] - data[:,0][0])
        be = data[:,0] - data[:,0][0]
        # print(be)
    f.close()
    all_BE[file.replace('_results.txt', '')] = be
    # with open('./outputs/BE/'+file, 'wb') as f:
    #     np.savetxt(f, be, delimiter=',')
        
    f.close()

In [3]:
all_BE_df = pd.DataFrame(all_BE).T

In [4]:
all_BE_df

,0,1,2
GRB081221,0.0,-0.406262,0.212556
GRB120119A,0.0,-0.670629,-0.162792
GRB210204A,0.0,4.174429,3.780217
GRB200613A,0.0,-1.128344,-0.477039
GRB171010A,0.0,4.031710,3.761295
GRB130925A,0.0,38.158142,46.187075
GRB180720B,0.0,4.841145,2.574501
GRB131231A,0.0,116.313657,106.218668
GRB131108A,0.0,8.541969,6.302999
GRB150514A,0.0,-1.242920,-0.304423


In [5]:
all_BE_df.sort_index(inplace=True, ascending=False)

In [6]:
all_BE_df

,0,1,2
GRB210619B,0.0,1.292045,4.151750
GRB210610B,0.0,0.314113,0.316729
GRB210204A,0.0,4.174429,3.780217
GRB201216C,0.0,-0.874223,-0.362957
GRB200829A,0.0,44.225721,26.898097
GRB200613A,0.0,-1.128344,-0.477039
GRB190114C,0.0,2.718761,2.147079
GRB180720B,0.0,4.841145,2.574501
GRB180703A,0.0,12.862939,1.249418
GRB171010A,0.0,4.031710,3.761295


In [7]:
grbparam = pd.read_csv(os.getcwd() + '/../data/GRBPARAM.csv', index_col=0).T

In [8]:
grbparam.sort_index(inplace=True, ascending=False)

In [9]:
grbparam

GRB,E0,Emax,redshift
GRB210619B,10.0,11000.0,1.9370
GRB210610B,30.0,380.0,1.1300
GRB210204A,10.0,400.0,0.8760
GRB201216C,15.0,700.0,1.1000
GRB200829A,25.0,3500.0,1.2500
GRB200613A,30.0,300.0,1.2200
GRB190114C,10.0,5000.0,0.4250
GRB180720B,25.0,17000.0,0.6540
GRB180703A,20.0,400.0,0.6678
GRB171010A,10.0,620.0,0.3285


In [10]:
new_df = pd.concat([grbparam, all_BE_df], axis=1)

In [11]:
new_df.drop(['Emax'], axis=1, inplace=True)

In [12]:
new_df

,E0,redshift,0,1,2
GRB210619B,10.0,1.9370,0.0,1.292045,4.151750
GRB210610B,30.0,1.1300,0.0,0.314113,0.316729
GRB210204A,10.0,0.8760,0.0,4.174429,3.780217
GRB201216C,15.0,1.1000,0.0,-0.874223,-0.362957
GRB200829A,25.0,1.2500,0.0,44.225721,26.898097
GRB200613A,30.0,1.2200,0.0,-1.128344,-0.477039
GRB190114C,10.0,0.4250,0.0,2.718761,2.147079
GRB180720B,25.0,0.6540,0.0,4.841145,2.574501
GRB180703A,20.0,0.6678,0.0,12.862939,1.249418
GRB171010A,10.0,0.3285,0.0,4.031710,3.761295


In [13]:
new_df.sort_index(inplace=True, ascending=False)

In [14]:
new_df

,E0,redshift,0,1,2
GRB210619B,10.0,1.9370,0.0,1.292045,4.151750
GRB210610B,30.0,1.1300,0.0,0.314113,0.316729
GRB210204A,10.0,0.8760,0.0,4.174429,3.780217
GRB201216C,15.0,1.1000,0.0,-0.874223,-0.362957
GRB200829A,25.0,1.2500,0.0,44.225721,26.898097
GRB200613A,30.0,1.2200,0.0,-1.128344,-0.477039
GRB190114C,10.0,0.4250,0.0,2.718761,2.147079
GRB180720B,25.0,0.6540,0.0,4.841145,2.574501
GRB180703A,20.0,0.6678,0.0,12.862939,1.249418
GRB171010A,10.0,0.3285,0.0,4.031710,3.761295


In [15]:
data = []
for i in os.listdir('./outputs/GOF/'):
    # with open('./outputs/GOF/' + i, 'rb') as f:
    #     data.append(np.load(f))
    if i.endswith('_GOF.txt'):
        data.append([i.replace('_GOF.txt', ''), *np.loadtxt('./outputs/GOF/' + i)])
df2 = pd.DataFrame(data, columns=['GRB', 'Null', 'Linear', 'Quadratic'])
df2.index = df2['GRB']
df2.drop('GRB', axis=1, inplace=True)
df2.sort_index(inplace=True, ascending=False)

In [16]:
final = pd.concat([new_df, df2], axis=1)

In [17]:
final

,E0,redshift,0,1,2,Null,Linear,Quadratic
GRB210619B,10.0,1.9370,0.0,1.292045,4.151750,4.030445,3.721603,3.249669
GRB210610B,30.0,1.1300,0.0,0.314113,0.316729,1.495271,1.418354,1.421608
GRB210204A,10.0,0.8760,0.0,4.174429,3.780217,10.197852,10.012142,10.231190
GRB201216C,15.0,1.1000,0.0,-0.874223,-0.362957,1.552029,1.298810,1.400843
GRB200829A,25.0,1.2500,0.0,44.225721,26.898097,11.241959,7.076828,9.211759
GRB200613A,30.0,1.2200,0.0,-1.128344,-0.477039,0.592548,0.691059,0.867639
GRB190114C,10.0,0.4250,0.0,2.718761,2.147079,2.728991,3.003190,2.025895
GRB180720B,25.0,0.6540,0.0,4.841145,2.574501,2.735034,1.071755,1.834891
GRB180703A,20.0,0.6678,0.0,12.862939,1.249418,11.257077,9.775528,12.142259
GRB171010A,10.0,0.3285,0.0,4.031710,3.761295,1.559373,0.689891,0.966344


In [19]:
final = final[['E0', 'redshift', 0, 'Null', 1, 'Linear', 2, 'Quadratic']]

In [21]:
final

,E0,redshift,0,Null,1,Linear,2,Quadratic
GRB210619B,10.0,1.9370,0.0,4.030445,1.292045,3.721603,4.151750,3.249669
GRB210610B,30.0,1.1300,0.0,1.495271,0.314113,1.418354,0.316729,1.421608
GRB210204A,10.0,0.8760,0.0,10.197852,4.174429,10.012142,3.780217,10.231190
GRB201216C,15.0,1.1000,0.0,1.552029,-0.874223,1.298810,-0.362957,1.400843
GRB200829A,25.0,1.2500,0.0,11.241959,44.225721,7.076828,26.898097,9.211759
GRB200613A,30.0,1.2200,0.0,0.592548,-1.128344,0.691059,-0.477039,0.867639
GRB190114C,10.0,0.4250,0.0,2.728991,2.718761,3.003190,2.147079,2.025895
GRB180720B,25.0,0.6540,0.0,2.735034,4.841145,1.071755,2.574501,1.834891
GRB180703A,20.0,0.6678,0.0,11.257077,12.862939,9.775528,1.249418,12.142259
GRB171010A,10.0,0.3285,0.0,1.559373,4.031710,0.689891,3.761295,0.966344


In [26]:
final['Null'] = final['Null'].round(1)
final['Linear'] = final['Linear'].round(1)
final['Quadratic'] = final['Quadratic'].round(1)
final[1] = final[1].round(1)
final[2] = final[2].round(1)

In [27]:
final

,E0,redshift,0,Null,1,Linear,2,Quadratic
GRB210619B,10.0,1.9370,0.0,4.0,1.3,3.7,4.2,3.2
GRB210610B,30.0,1.1300,0.0,1.5,0.3,1.4,0.3,1.4
GRB210204A,10.0,0.8760,0.0,10.2,4.2,10.0,3.8,10.2
GRB201216C,15.0,1.1000,0.0,1.6,-0.9,1.3,-0.4,1.4
GRB200829A,25.0,1.2500,0.0,11.2,44.2,7.1,26.9,9.2
GRB200613A,30.0,1.2200,0.0,0.6,-1.1,0.7,-0.5,0.9
GRB190114C,10.0,0.4250,0.0,2.7,2.7,3.0,2.1,2.0
GRB180720B,25.0,0.6540,0.0,2.7,4.8,1.1,2.6,1.8
GRB180703A,20.0,0.6678,0.0,11.3,12.9,9.8,1.2,12.1
GRB171010A,10.0,0.3285,0.0,1.6,4.0,0.7,3.8,1.0


In [28]:
final.to_latex("./outputs/table.tex", float_format="%.2f")